In [ ]:
import os
from dotenv import load_dotenv
import textwrap
import bs4
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    raise Exception("Please set the OPENAI_API_KEY environment variable.")

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
vector_store = InMemoryVectorStore(embeddings)

In [7]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
_ = vector_store.add_documents(documents=all_splits)

In [9]:
# pull prompt template from hub
prompt = hub.pull("rlm/rag-prompt")

/Users/joypaulsen/Library/Caches/pypoetry/virtualenvs/rag-experiments-LTBInUdO-py3.13/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# state managed by the graph
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [12]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(textwrap.fill(response["answer"], width=80))

Task Decomposition is the process of breaking down a complex task into smaller,
manageable steps or sub-tasks, allowing for more efficient problem-solving.
Techniques like Chain of Thought (CoT) and Tree of Thoughts facilitate this by
prompting models to think step by step or explore multiple reasoning
possibilities. This approach enhances the model's performance and provides
clearer insights into its reasoning process.
